# What is the advantage of `async await` over `ContinueWith` in C#?

## Using `ContinueWith` makes callback hell; using `async await` removes callback hell and makes program readable

In [1]:
using System;
using System.Threading;
using System.Threading.Tasks;

public class Turkey
{
    public Task Cook()
    {
        return Task.Run(() =>
        {
            Task.Delay(2000).Wait(); // Simulate cooking time
            Console.WriteLine("Cooking turkey...");
        });
    }
}

public class Gravy
{
    public Task Cook()
    {
        return Task.Run(() =>
        {
            Task.Delay(1000).Wait(); // Simulate cooking time
            Console.WriteLine("Cooking gravy...");
        });
    }
}

public class Vegetables
{
    public Task Cook()
    {
        return Task.Run(() =>
        {
            Task.Delay(1500).Wait(); // Simulate cooking time
            Console.WriteLine("Cooking vegetables...");
        });
    }
}

public class Program
{
    public static void Main()
    {
        Console.WriteLine("Cooking Started");

        var turkey = new Turkey();

        // Start cooking the turkey
        turkey.Cook().ContinueWith(turkeyTask =>
        {
            if (turkeyTask.IsCompletedSuccessfully)
            {
                Console.WriteLine("Turkey is cooked!");

                var gravy = new Gravy();
                // Start cooking the gravy
                gravy.Cook().ContinueWith(gravyTask =>
                {
                    if (gravyTask.IsCompletedSuccessfully)
                    {
                        Console.WriteLine("Gravy is cooked!");

                        var vegetables = new Vegetables();
                        // Start cooking the vegetables
                        vegetables.Cook().ContinueWith(vegetablesTask =>
                        {
                            if (vegetablesTask.IsCompletedSuccessfully)
                            {
                                Console.WriteLine("Vegetables are cooked!");
                                Console.WriteLine("Dinner is ready!");
                            }
                            else
                            {
                                Console.WriteLine("Failed to cook vegetables.");
                            }
                        });
                    }
                    else
                    {
                        Console.WriteLine("Failed to cook gravy.");
                    }
                });
            }
            else
            {
                Console.WriteLine("Failed to cook turkey.");
            }
        });

        // Block the main thread for a while so the program doesn't end
        // and we can see the output of the cooking tasks in the Console
        Thread.Sleep(10000);
    }
}
Program.Main();


Cooking Started
Cooking turkey...
Turkey is cooked!
Cooking gravy...
Gravy is cooked!
Cooking vegetables...
Vegetables are cooked!
Dinner is ready!


Callback hell in the above example; with `async await` more readable

In [2]:
public static async Task Main()
{
    Console.WriteLine("Cooking Started");

    var turkey = new Turkey();
    await turkey.Cook();
    Console.WriteLine("Turkey is cooked!");

    var gravy = new Gravy();
    await gravy.Cook();
    Console.WriteLine("Gravy is cooked!");

    var vegetables = new Vegetables();
    await vegetables.Cook();
    Console.WriteLine("Vegetables are cooked!");

    Console.WriteLine("Dinner is ready!");
}
await Main();

Cooking Started
Cooking turkey...
Turkey is cooked!
Cooking gravy...
Gravy is cooked!
Cooking vegetables...
Vegetables are cooked!
Dinner is ready!


## `ContinueWith` always schedules continuation, even if task fails or cancelled.



Need to always check task state (i.e. `task.isCompletedSuccessfully` call in the above example).

With `async await`, if task fails or is cancelled, will not schedule the continuation

## Thread management; with `ContinueWith`, the continuation runs on a thread pool thread, not necessarily on the same thread as caller

To force it need to explicitly specify like example below

In [3]:
turkey.Cook()
    .ContinueWith(_ =>
    {
        var gravy = new Gravy();
        gravy.Cook();
    }, TaskScheduler.FromCurrentSynchronizationContext());

Error: (1,1): error CS0103: The name 'turkey' does not exist in the current context

But if we use `async await`, it will, if the app is a UI app, ensure the continuation is run on the calling thread

# What is the issue with using `Wait()` or `Result()`?

It blocks the calling thread